In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [3]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > DESIRED_ACCURACY):
            print('Reached 99.9% accuracy so cancelling training!')
            self.model.stop_training = True
callbacks = myCallback()

--2020-11-22 16:17:26--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.03s   

2020-11-22 16:17:26 (96.2 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [4]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['accuracy'])

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory('/tmp/h-or-s', 
                                                        batch_size=10, 
                                                        target_size=(150, 150), 
                                                        class_mode='binary')


Found 80 images belonging to 2 classes.


In [6]:
history = model.fit_generator(train_generator, 
                                  steps_per_epoch=8,
                                  epochs=15,
                                  callbacks=[callbacks],
                                  verbose=1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
8/8 [==============================] - 2s 245ms/step - loss: 1.7992 - accuracy: 0.4375
Epoch 2/15
8/8 [==============================] - 2s 240ms/step - loss: 0.5181 - accuracy: 0.8250
Epoch 3/15
8/8 [==============================] - 2s 239ms/step - loss: 0.3441 - accuracy: 0.8500
Epoch 4/15
8/8 [==============================] - 2s 239ms/step - loss: 0.2270 - accuracy: 0.9250
Epoch 5/15
8/8 [==============================] - 2s 243ms/step - loss: 0.1400 - accuracy: 0.9500
Epoch 6/15
8/8 [==============================] - 2s 239ms/step - loss: 0.1572 - accuracy: 0.9500
Epoch 7/15
8/8 [==============================] - 2s 240ms/step - loss: 0.0884 - accuracy: 0.9875
Epoch 8/15
8/8 [==============================] - 2s 240ms/step - loss: 0.0995 - accuracy: 0.9500
Epoch 9/15
8/8 [==============================] - 2s 241ms/step - loss: 0.0665 - accuracy: 0.9750
Epoch 10/15
8/8 [=========================